In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q git+https://github.com/snap-stanford/deepsnap.git

2.0.1+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 19.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done


In [ ]:
import tensorflow.compat.v1 as tf
import random
import pandas as pd
import torch_scatter
import torch.nn as nn
from torch.nn import Linear, Sequential, LayerNorm, ReLU
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.data import DataLoader

import numpy as np
import time
import torch.optim as optim
from tqdm import trange
import pandas as pd
import copy
import matplotlib.pyplot as plt
import os
import sys


In [ ]:
import tensorflow.compat.v1 as tf
import functools
import json
from torch_geometric.data import Data
import enum
import itertools

In [ ]:
def triangles_to_edges(faces):
  """Computes mesh edges from triangles.
     Note that this triangles_to_edges method was provided as part of the
     code release for the MeshGraphNets paper by DeepMind, available here:
     https://github.com/deepmind/deepmind-research/tree/master/meshgraphnets
  """
  # collect edges from triangles
  edges = tf.concat([faces[:, 0:2],
                     faces[:, 1:3],
                     tf.stack([faces[:, 2], faces[:, 0]], axis=1)], axis=0)
  # those edges are sometimes duplicated (within the mesh) and sometimes
  # single (at the mesh boundary).
  # sort & pack edges as single tf.int64
  receivers = tf.reduce_min(edges, axis=1)
  senders = tf.reduce_max(edges, axis=1)
  packed_edges = tf.bitcast(tf.stack([senders, receivers], axis=1), tf.int64)
  # remove duplicates and unpack
  unique_edges = tf.bitcast(tf.unique(packed_edges)[0], tf.int32)
  senders, receivers = tf.unstack(unique_edges, axis=1)
  # create two-way connectivity
  return (tf.concat([senders, receivers], axis=0),
          tf.concat([receivers, senders], axis=0))

from pandas._libs import internals
class NodeType(enum.IntEnum):

    internal = 0
    upper_lower = 1
    SIZE = 2

In [ ]:
data_file="/content/drive/NeuralOps/example_csv_data"

data_=os.listdir(data_file)
len(data_)

3

In [ ]:
import warnings
warnings.filterwarnings("ignore")


def create_dataset(data_file):
    data_list=[]
    file_name_list=[]
    file_list=os.listdir(data_file)
    j=1
    for item in file_list:
        design_name=item.rpartition('_')[0]
        if design_name not in file_name_list and design_name != ".DS" :
            print(j)
            #info=design_name.split("_")
            #extra_features=torch.tensor([float(info[-5])*1e-9, float(info[-3])*1e-3, float(info[-1])*np.pi/180])
            #extra_features=torch.tensor([float(info[-5])*1e-11, float(info[-3])*1e-5])

            file_name_list.append(design_name)

            design_node=data_file+'/'+design_name+'_Nodes.csv'
            design_element=data_file+'/'+design_name+'_elements.csv'

            df = pd.read_csv(design_node)
            df_node_indices_four=pd.read_csv(design_element)

            # four node 3D element to four 3nodel faces

            df_node_indices_three = pd.DataFrame( columns = ['elem1','elem2','elem3'])
            df_node_indices_three2 = pd.DataFrame( columns = ['elem1','elem2','elem3'])


            for i in range(df_node_indices_four.shape[0]):
                four_nodes=df_node_indices_four[i:i+1].values.tolist()[0][-4:]

                three_nodes=np.array(list(itertools.combinations(four_nodes,3)))
                cell_df=pd.DataFrame(three_nodes, columns = ['elem1','elem2','elem3'])
                df_node_indices_three = df_node_indices_three.append(cell_df)

                three_nodes2=np.array(list(itertools.permutations(four_nodes,3)))
                cell_df2=pd.DataFrame(three_nodes2, columns = ['elem1','elem2','elem3'])
                df_node_indices_three2 = df_node_indices_three2.append(cell_df2)



            cells_index2=np.vstack((df_node_indices_three2["elem1"].to_numpy(),df_node_indices_three2["elem2"].to_numpy(),df_node_indices_three2["elem3"].to_numpy())).T.astype('int32')
            cells2=torch.tensor(cells_index2)

            # cells (element indices)
            cells_index=np.vstack((df_node_indices_three["elem1"].to_numpy(),df_node_indices_three["elem2"].to_numpy(),df_node_indices_three["elem3"].to_numpy())).T.astype('int32')
            cells=torch.tensor(cells_index)

            # node positions
            node_position=np.vstack((df["x"].to_numpy(),df["y"].to_numpy(),df["z"].to_numpy())).T.astype('float32')
            mesh_pos=torch.tensor(node_position)

            # disp, strain, stress information
            stress=torch.tensor(np.vstack((df["S11"].to_numpy(),df["S22"].to_numpy(),df["S33"].to_numpy(),df["S12"].to_numpy(),df["S13"].to_numpy(),df["S12.1"].to_numpy())).T.astype('float32'))
            stress_mises=torch.unsqueeze(torch.tensor(df["mises"].to_numpy().T.astype('float32')), 1)
            stress11=torch.unsqueeze(torch.tensor(df["S11"].to_numpy().T.astype('float32')), 1)
            stress22=torch.unsqueeze(torch.tensor(df["S22"].to_numpy().T.astype('float32')), 1)
            stress33=torch.unsqueeze(torch.tensor(df["S33"].to_numpy().T.astype('float32')), 1)
            stress12=torch.unsqueeze(torch.tensor(df["S12"].to_numpy().T.astype('float32')), 1)
            stress13=torch.unsqueeze(torch.tensor(df["S13"].to_numpy().T.astype('float32')), 1)
            stress23=torch.unsqueeze(torch.tensor(df["S12.1"].to_numpy().T.astype('float32')), 1)


            # get edge indices
            edges2 = triangles_to_edges(tf.convert_to_tensor(cells_index))
            edge_index2 = torch.cat( (torch.tensor(edges2[0].numpy()).unsqueeze(0) ,
                         torch.tensor(edges2[1].numpy()).unsqueeze(0)), dim=0).type(torch.long)

            # get new connectivity, connect all noded within the distance D
            D=0.05
            new_connectivity=df.iloc[:,-len(df):].values
            edge_index=torch.tensor(np.where((new_connectivity<= D) & (new_connectivity>0)))


            # get edge features
            u_i=torch.tensor(node_position)[edge_index[0]]
            u_j=torch.tensor(node_position)[edge_index[1]]
            u_ij=u_i-u_j
            u_ij_norm = torch.norm(u_ij,p=2,dim=1,keepdim=True)
            edge_attr = torch.cat((u_ij,u_ij_norm),dim=-1).type(torch.float)

            # get input x
            node_type_info=df["nodetype"].to_numpy().T.astype('int32')
            node_type = torch.tensor(np.array(tf.one_hot(tf.convert_to_tensor(node_type_info), NodeType.SIZE))).squeeze(1)
            x_without_features = torch.cat((mesh_pos,node_type),dim=-1).type(torch.float)

            #extra_features_stack=extra_features.repeat(x_without_features.shape[0],1)
            #x_=torch.cat((mesh_pos,extra_features_stack),dim=-1).type(torch.float)
            #x=torch.cat((x_,node_type),dim=-1).type(torch.float)
            #x = x_without_features
            j=j+1
            # get data ist
            data_list.append(Data(x=x_without_features, x_without_features=x_without_features , edge_index=edge_index,
                                  edge_index2=edge_index2, edge_attr=edge_attr, stress=stress,
                                  cells=cells, cells2=cells2,mesh_pos=mesh_pos, stress11=stress11,
                                 stress22=stress22, stress33=stress33, stress12=stress12,
                                 stress13=stress13, stress23=stress23,stress_mises=stress_mises, name=design_name ))
        else:
            pass

    print("Done collecting data!")


    return torch.save(data_list, os.path.join("/content/drive/NeuralOps/" + 'example_data_NO.pt'))


create_dataset(data_file)

1
Done collecting data!
